In [1]:
%load_ext autoreload
%autoreload 2

# Text Summarizer

In [2]:
import re
from sentence_transformers import SentenceTransformer
from summary.pacsum_extractor import PacSumExtractor, HParams
from summary.detr_extractor import DETRExtractor, ArgType
from summary import utils
from datasets.processing_funcs import PIPELINE
from datasets.fbp_dataset import load_only_texts, FBPSummaryDataset

from types import SimpleNamespace
import torch
import numpy as np

sentence_separators = re.compile("[.?!]")

utils.pd.options.display.float_format = "{:.3f}".format

c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\Zumo\anaconda3\envs\torch-gpu\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def tuning_metric(df):
    return df["mean"]["f"]

In [4]:
texts = load_only_texts("./input/feedback-prize-2021/summary/original/", PIPELINE, 1)
sums = load_only_texts("./input/feedback-prize-2021/summary/references/", PIPELINE, 1)
dataset = FBPSummaryDataset(texts, sums)


Loading Dataset: 100%|██████████| 10/10 [00:00<00:00, 2494.53it/s]


# PACSUM

In [5]:
pacsum = PacSumExtractor(
    sentence_transformer=SentenceTransformer("all-mpnet-base-v2"),
    extract_num=3,
)


In [6]:
pacsum.tune_hparams(dataset, metrics_keys=tuning_metric)


100%|██████████| 10/10 [00:02<00:00,  3.65it/s]


Best: 0.485 - HParams(beta=0.00, lambda1=0.00, lambda2=1.00)
Best: 0.485 - HParams(beta=0.00, lambda1=0.11, lambda2=0.89)
Best: 0.521 - HParams(beta=0.00, lambda1=0.44, lambda2=0.56)
Best: 0.525 - HParams(beta=0.00, lambda1=0.78, lambda2=0.22)
Best: 0.560 - HParams(beta=0.11, lambda1=0.44, lambda2=0.56)
Best: 0.610 - HParams(beta=0.22, lambda1=0.44, lambda2=0.56)
------------------------------
The best hyper-parameter:  HParams(beta=0.22, lambda1=0.44, lambda2=0.56)
The best rouge score    :  0.610
   rouge-1  rouge-2  rouge-l  mean
r    0.617    0.503    0.597 0.572
p    0.717    0.586    0.693 0.666
f    0.658    0.536    0.636 0.610


HParams(beta=0.22, lambda1=0.44, lambda2=0.56)

# DETR

In [7]:
detr = DETRExtractor(SimpleNamespace(
    # Model parameters
    hidden_dim=2048,
    num_queries=40,
    class_depth=1,
    bbox_depth=3,
    resume=None,
    init_last_biases=True,
    dropout=0,
    init_weight=None,
    pretrained=True,
    frozen_weights="./checkpoint.pth",
    glob_attn_words=True,
    # Other parameters
    num_classes=7,
    device="cuda",
))


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
detr.tune_hparams(dataset, metrics_keys=tuning_metric)

100%|██████████| 10/10 [00:05<00:00,  1.73it/s]


Best: 0.424 - [ArgType.Claim]
Best: 0.455 - [ArgType.ConcludingStatement]
Best: 0.460 - [ArgType.Lead]
Best: 0.478 - [ArgType.Position]
Best: 0.564 - [ArgType.ConcludingStatement, ArgType.Lead]
Best: 0.568 - [ArgType.ConcludingStatement, ArgType.Lead, ArgType.Position]
------------------------------
The best hyper-parameter:  [ArgType.ConcludingStatement, ArgType.Lead, ArgType.Position]
The best rouge score    :  0.568
   rouge-1  rouge-2  rouge-l  mean
r    0.773    0.629    0.743 0.715
p    0.540    0.400    0.520 0.486
f    0.626    0.477    0.602 0.568


[ArgType.ConcludingStatement, ArgType.Lead, ArgType.Position]

# CASCADE

In [9]:
class MixExtract:
    def extract_summary(self, text: str) -> str:
        text_sum = detr.extract_summary(text)
        return pacsum.extract_summary(text_sum)

cascade = MixExtract()

# Test

In [14]:
text, info = dataset[2]
print(text)
print()
print(info["summary"])

def test_extractor(extractor):
    summ = extractor.extract_summary(text)
    print(summ)

    res, metrics = utils.extract_summary_iterator(extractor, dataset)
    print("Mean length of summaries:", np.mean([len(sentence_separators.split(r)) - 1 for r in res]))

    print(f"Mean Rouge f score: {tuning_metric(metrics):.3f}")
    return metrics

the driverless cars are a bad idea. in theroy, they sound wonderful. but in reality they are a safty hazard. to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and are still are not legal in some states around the u.s.the driver has to stay aware of the other things around them. even thought you still have to keep your hands on the wheel because of the touch sensors. the article states, in fact, none of the cars developed so far are completely driverless. this statment shows there is still room for human error in these driverless cars.even though human error is still involved, these cars still can't go through construction areas or around wrecks but they still alert you when you come across it.the prompt states, but all are designed when the road requires human skills, such as navigating through work zones and around acciedents. this means the human driver must remain alert and be ready to take over when the situation requires. peopl

In [15]:
test_extractor(pacsum)

to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and are still are not legal in some states around the u. these driverless cars are a hazard. this statment shows there is still room for human error in these driverless cars.


100%|██████████| 10/10 [00:00<00:00, 11.45it/s]

Mean length of summaries: 3.0
Mean Rouge f score: 0.610


,rouge-1,rouge-2,rouge-l,mean
r,0.617,0.503,0.597,0.572
p,0.717,0.586,0.693,0.666
f,0.658,0.536,0.636,0.610


In [16]:
test_extractor(detr)

the driverless cars are a bad idea. in theroy, they sound wonderful. but in reality they are a safty hazard. to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and are still are not legal in some states around the u.s.the way they seem to undertested amd people don't want to trust them. manly because you would still have to pay attention to the road, still have to drive through construction and wrecks, and are technically still not fully legalised in some states like california, nevada and florida. with a bit more field testing these driverless cars could be one of our


100%|██████████| 10/10 [00:04<00:00,  2.46it/s]

Mean length of summaries: 5.9
Mean Rouge f score: 0.568


,rouge-1,rouge-2,rouge-l,mean
r,0.773,0.629,0.743,0.715
p,0.540,0.400,0.520,0.486
f,0.626,0.477,0.602,0.568


In [17]:
test_extractor(cascade)

to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and are still are not legal in some states around the u. the driverless cars are a bad idea. manly because you would still have to pay attention to the road, still have to drive through construction and wrecks, and are technically still not fully legalised in some states like california, nevada and florida.


100%|██████████| 10/10 [00:04<00:00,  2.30it/s]

Mean length of summaries: 2.9
Mean Rouge f score: 0.569


,rouge-1,rouge-2,rouge-l,mean
r,0.593,0.453,0.563,0.536
p,0.678,0.529,0.647,0.618
f,0.627,0.483,0.597,0.569


In [20]:
pacsum.extract_summary(
    "the driverless cars are a bad idea. in theroy, they sound wonderful. but in reality they are a safty hazard. "
    "to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and "
    "are still are not legal in some states around the u.s.the way they seem to undertested amd people don't want "
    "to trust them. manly because you would still have to pay attention to the road, still have to drive through "
    "construction and wrecks, and are technically still not fully legalised in some states like california, nevada "
    "and florida. with a bit more field testing these driverless cars could be one of our greatest achivements."
)


'to start with the driver needs to stay aware of the car and traffic, they cant go through construction, and are still are not legal in some states around the u. the driverless cars are a bad idea. with a bit more field testing these driverless cars could be one of our greatest achivements.'